In [1]:
#importing required libraries

import numpy as np

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
# image data generator

train_datagen = ImageDataGenerator(rescale=1./255.,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode="nearest",
                                  preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory("dog vs cat/dataset/train",
                                                 target_size=(150,150),
                                                 color_mode="rgb",
                                                 batch_size=16,
                                                 class_mode="categorical")

Found 2000 images belonging to 2 classes.


In [5]:
valid_datagen = ImageDataGenerator(rescale=1./255.,
                                  preprocessing_function=preprocess_input)

valid_generator = valid_datagen.flow_from_directory("dog vs cat/dataset/valid",
                                                 target_size=(150,150),
                                                 color_mode="rgb",
                                                 batch_size=16,
                                                 class_mode="categorical")

Found 2000 images belonging to 2 classes.


## Model

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [8]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150,150,3))

In [9]:
model_tl = Sequential()
model_tl.add(base_model)
model_tl.add(Flatten())

model_tl.add(Dense(256, activation="relu"))
model_tl.add(Dropout(0.5))
model_tl.add(Dense(128, activation="relu"))

model_tl.add(Dense(2, activation="softmax"))

In [10]:
model_tl.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 16,845,250
Trainable params: 16,845,250
Non-trainable params: 0
________________________________________________________________

In [11]:
# freeze early layers
base_model.trainable = False

In [12]:
# optimizer
from keras import optimizers
opt = optimizers.Adam(lr=0.001)

In [13]:
# compiling
model_tl.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [14]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_valid = valid_generator.n//valid_generator.batch_size

In [15]:
# training

model_tl.fit_generator(
        train_generator,
        steps_per_epoch=step_size_train,
        epochs=20,
        validation_data=valid_generator,
        validation_steps=step_size_valid)

Epoch 1/20
125/125 [==============================] - 1025s 8s/step - loss: 0.6079 - acc: 0.7120 - val_loss: 0.3098 - val_acc: 0.8670
Epoch 2/20
125/125 [==============================] - 1000s 8s/step - loss: 0.4434 - acc: 0.7910 - val_loss: 0.2735 - val_acc: 0.8800
Epoch 3/20
125/125 [==============================] - 1000s 8s/step - loss: 0.3863 - acc: 0.8320 - val_loss: 0.2893 - val_acc: 0.8715
Epoch 4/20
125/125 [==============================] - 992s 8s/step - loss: 0.4003 - acc: 0.8135 - val_loss: 0.2612 - val_acc: 0.8845
Epoch 5/20
125/125 [==============================] - 988s 8s/step - loss: 0.3563 - acc: 0.8380 - val_loss: 0.2763 - val_acc: 0.8880
Epoch 6/20
125/125 [==============================] - 988s 8s/step - loss: 0.3468 - acc: 0.8410 - val_loss: 0.2869 - val_acc: 0.8810
Epoch 7/20
125/125 [==============================] - 989s 8s/step - loss: 0.3460 - acc: 0.8395 - val_loss: 0.2562 - val_acc: 0.8970
Epoch 8/20
125/125 [==============================] - 986s 8s/step